In [ ]:
import numpy as np
import astropy.constants as con
import astropy.units as u
import matplotlib.pyplot as plt
import scipy.stats as st
import simulation_support as simsup
import importlib
import astropy.visualization as viz

from yaff import common_models as cm
from yaff.fitting import Parameter
from yaff import fitting

%matplotlib qt
# plt.style.use('nice.mplstyle')

In [ ]:
total_time = 20 << u.s
time_bin = 0.1 << u.s

time_bins = np.arange(0, (total_time + time_bin).to_value(u.s), time_bin.to_value(u.s)) << u.s
time_mids = time_bins[:-1] + np.diff(time_bins)/2

num_time_bins = time_mids.size

In [ ]:
num_injections = 30
injection_rise = 1
injection_fall = 4
# injection_duration = 1 << u.s

rng = np.random.default_rng()
injection_time_indices = rng.integers(0, num_time_bins - 1, size=num_injections)
print(injection_time_indices)

# The spectral index starts linearly going up
indices = simsup.linear_slew(7, 5, num_time_bins)

efluxes = simsup.linear_slew(0.4, 3, num_time_bins)

# The spectral indices and electron fluxes spike
# at the same time: beamlike injection of electrons
for it in injection_time_indices:
    cur_index = indices[it] 
    injection_index_change = 0.4
    index_delta = 0.3
    indices -= (s := simsup.spike(
        time_mids.to_value(u.s),
        time_mids[it].to_value(u.s),
        injection_rise,
        injection_fall,
        index_delta
    ))

    cur_flux = efluxes[it]
    flux_delta = 2
    # efluxes += s * (flux_delta / index_delta)
    efluxes += simsup.spike(
        time_mids.to_value(u.s),
        time_mids[it].to_value(u.s),
        injection_rise,
        injection_fall,
        flux_delta
    )

efluxes[efluxes < 0] = 0
# efluxes /= 10

index_cut = 2.1
indices[indices < index_cut] = index_cut

In [ ]:
import scipy.signal as sig

critical_freq = 1 / (1 << u.s)
sample_rate = (1 / time_bin)
order = 5

butterb, buttera = sig.butter(
    N=order,
    Wn=critical_freq.to_value(u.Hz),
    btype='lowpass',
    # Data is already sampled at 0.5s cadence,
    # so we want a digital filter (not analog)
    # see: https://dsp.stackexchange.com/a/40715
    analog=False,
    output='ba',
    fs=sample_rate.to_value(u.Hz)
)

low_idx = sig.filtfilt(butterb, buttera, indices, method='gust')
low_eflux = sig.filtfilt(butterb, buttera, efluxes, method='gust')

In [ ]:
fig, ax = plt.subplots()
ax.stairs(indices, time_bins.value, label='spectral index')
ax.stairs(low_idx, time_bins.value, label='spectral index (lowpass)')
ax.stairs(efluxes, time_bins.value, label='electron flux')
ax.stairs(low_eflux, time_bins.value, label='electron flux (lowpass)')
# lab = 'injection'
# for idx in injection_time_indices:
    # ax.axvline(time_mids[idx].value, color='red', alpha=0.6, zorder=-1, label=lab)
    # Clear the label so it only shows up once
    # lab = None
ax.legend()
plt.show()

In [ ]:
# A poor-man's Neupert effect
temp_start = 13 << u.MK
fudge = 0.3
temp_shape = indices.cumsum()
temp_shape /= temp_shape.max()
temp_shape *= fudge
temp_shape += 1
temps = temp_shape * temp_start


# The emission measure is the cumulative effect of
# heating (??)
em_start, em_end = (5, 8) << (1e48 * u.cm**-3)
em_shape = temp_shape.cumsum()
em_shape /= em_shape.max()
ems = (1 + em_shape) * em_start


fig, ax = plt.subplots()
with viz.quantity_support():
    ax.stairs(temps, time_bins, color='orange')
    axx = ax.twinx()
    axx.stairs(ems, time_bins, color='blue')

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.stairs(low_idx, time_bins.value, label='spectral index (lowpass)', color='blue')
axx = ax.twinx()
axx.stairs(temps, time_bins.value, label='temperatures', color='orange')

ax.legend()
ax.set(ylabel='spectral index')
axx.set(ylabel='temperature')
axx.legend()
plt.show()

In [ ]:
# The nonthermal emission should extend to rather low energies
cutoff_energies = (3 << u.keV) * np.ones_like(time_mids.value)

In [ ]:
energy_bins = np.geomspace(4, 200, num=32) << u.keV
exposure = total_time
de = np.diff(energy_bins) << u.keV
energy_mids = (energy_bins[:-1] + np.diff(energy_bins)/2) << u.keV

In [ ]:
# Effective area of 10cm2 with 100um of Al
geometric_area = 10 << u.cm**2
response_vector = (u.ct / u.ph) * geometric_area * np.ones(energy_bins.size - 1)#[1.56e-03, 6.77e-03, 2.30e-02, 6.39e-02, 1.51e-01, 3.09e-01, 5.62e-01, 9.26e-01, 1.40e+00, 1.97e+00, 2.62e+00, 3.31e+00, 4.03e+00, 4.73e+00, 5.40e+00, 6.03e+00, 6.60e+00, 7.11e+00, 7.56e+00, 7.94e+00, 8.28e+00, 8.57e+00, 8.81e+00, 9.01e+00, 9.18e+00, 9.33e+00, 9.44e+00, 9.54e+00, 9.62e+00, 9.69e+00, 9.74e+00, 9.79e+00, 9.83e+00, 9.86e+00, 9.88e+00, 9.90e+00, 9.92e+00, 9.93e+00, 9.95e+00, 9.96e+00, 9.96e+00, 9.97e+00, 9.97e+00, 9.98e+00, 9.98e+00, 9.99e+00, 9.99e+00, 9.99e+00, 9.99e+00, 9.99e+00, 9.99e+00, 9.99e+00, 1.00e+01, 1.00e+01, 1.00e+01, 1.00e+01, 1.00e+01, 1.00e+01, 1.00e+01,] << (u.cm**2 * u.ct / u.ph)

In [ ]:
thermal_truth = np.zeros((time_mids.size, energy_bins.size - 1)) << u.ct
nonthermal_truth = np.zeros((time_mids.size, energy_bins.size - 1)) << u.ct

# total_em = em_start
# warm_target_size = 4 << u.Mm
for i in range(cutoff_energies.size):
    parameters = {
        'temperature': temps[i],
        'emission_measure': ems[i],

        'cutoff_energy': cutoff_energies[i],
        'spectral_index': low_idx[i] << u.one,
        'electron_flux': (float(low_eflux[i]) << (1e35 * u.electron / u.s)),

        # 'loop_segment_density': 1 << (1e11 * u.cm**-3),
        # 'loop_segment_length': 4 << u.Mm
    }

    # _, add_em = warm_target_parameters(
    #     density=parameters['loop_segment_density'],
    #     temperature=parameters['temperature'],
    #     low_energy_cutoff=parameters['cutoff_energy'],
    #     segment_length=parameters['loop_segment_length'],
    #     electron_flux=parameters['electron_flux']
    # )
    # total_em += add_em

    parameters = {k: Parameter(v, True) for (k, v) in parameters.items()}
    args = {
        'parameters': parameters,
        'photon_energy_edges': energy_bins.to_value(u.keV)
    }
    thermal_truth[i] = response_vector * ((cm.thermal(args) << (u.ph / u.cm**2 / u.keV / u.s)) * time_bin * de)
    # nonthermal_truth[i] = response_vector * ((warm_thick_target(args) << (u.ph / u.keV / u.cm**2 / u.s)) * time_bin * de)
    nonthermal_truth[i] = response_vector * ((cm.thick_target(args) << (u.ph / u.keV / u.cm**2 / u.s)) * time_bin * de)

thermal_truth[thermal_truth < 0] = 0
nonthermal_truth[nonthermal_truth < 0] = 0

thermal_truth = thermal_truth.astype(int)
nonthermal_truth = nonthermal_truth.astype(int)

def full_spectrogram():
    return thermal_truth + nonthermal_truth

In [ ]:
import matplotlib.colors as mcol

unit = u.ct

fig, ax = plt.subplots()
norm = mcol.LogNorm()
cmap = plt.get_cmap('plasma').copy()
ax.pcolormesh(
    time_bins.to_value(u.s),
    energy_bins,
    full_spectrogram().T.to_value(unit).astype(int),
    norm=norm,
    cmap=cmap
)
ax.set(yscale='log', xlabel='time (s)', ylabel='energy (keV)')
plt.show()

In [ ]:
import matplotlib.colors as mcol

unit = u.ct

fig, ax = plt.subplots()
norm = mcol.LogNorm()
cmap = plt.get_cmap('plasma').copy()
ax.pcolormesh(
    time_bins.to_value(u.s),
    energy_bins,
    nonthermal_truth.T.to_value(unit).astype(int),
    norm=norm,
    cmap=cmap
)
ax.set(yscale='log', xlabel='time (s)', ylabel='energy (keV)')
plt.show()

In [ ]:
closest = lambda a, v: np.argmin(np.abs(a - v))

'''
Let's say we have a Ba133 source on board.
For X-rays that's about 2e5 count/second.
There will be lines at 4 keV, 31 keV, and 81 keV
'''
noise = np.ones_like(thermal_truth[0].value).astype(float)

baseline_rate = 10 << u.Hz
baseline_cts = (baseline_rate * time_bin * num_time_bins)
noise *= baseline_cts

line1_rate = 200 << u.Hz
line1_cts = (line1_rate * time_bin * num_time_bins)
noise[closest(energy_mids, 6 << u.keV):closest(energy_mids, 7 << u.keV)] = line1_cts

line2_rate = 100 << u.Hz
line2_cts = (line2_rate * time_bin * num_time_bins)
noise[closest(energy_mids, 29 << u.keV):closest(energy_mids, 32 << u.keV)] = line2_cts

# Quieter between lines at high energy
noise[closest(energy_mids, 35 << u.keV):closest(energy_mids, 100 << u.keV)] /= 3
noise[closest(energy_mids, 8 << u.keV):closest(energy_mids, 28 << u.keV)] /= 3
noise[closest(energy_mids, 80 << u.keV):closest(energy_mids, 84 << u.keV)] *= 5
noise[closest(energy_mids, 120 << u.keV):] *= 4

# Given the average count rate, we expect a certain amount of noise
# rate = noise / num_time_bins
# noise_spectrogram = st.poisson.rvs(np.tile(rate, (num_time_bins, 1))) << u.ct
noise_spectrogram = np.empty_like(thermal_truth)
p = [1 / num_time_bins] * num_time_bins
for (i, n) in enumerate(noise):
    noise_spectrogram[:, i] = st.multinomial.rvs(n, p) << u.ct
# n = noise[0]
# st.multinomial.rvs(n, p).sum()

In [ ]:
num = 1e4
bins = int(1e3)
p = bins * [1 / bins]
test = st.multinomial.rvs(n=num, p=p)

In [ ]:
import matplotlib.colors as mcol

unit = u.ct

fig, ax = plt.subplots()
norm = mcol.LogNorm()
cmap = plt.get_cmap('plasma').copy()
ax.pcolormesh(
    time_bins.to_value(u.s),
    energy_bins,
    noise_spectrogram.value.T,
    norm=norm,
    cmap=cmap
)
ax.set(yscale='log', xlabel='time (s)', ylabel='energy (keV)', title='background counts')
plt.show()

In [ ]:
import fathon
from fathon import fathonUtils as fu

a = fu.subtractMean(test) #fu.toAggregated(test)
dfa = fathon.DFA(a)
min_window = 4
windows = np.arange(min_window, a.size)
n, f = dfa.computeFlucVec(windows, revSeg=True, polOrd=2)
h, h_intercept = dfa.fitFlucVec()
h

In [ ]:
mean = (thermal_truth + nonthermal_truth + noise_spectrogram).to_value(u.ct).astype(int)
data = st.poisson.rvs(mean) << u.ct
systematic = 0.1
data_err = np.sqrt(data.to_value(u.ct) + (systematic * data.to_value(u.ct))**2) << u.ct

data[data < 0] = 0

In [ ]:
import matplotlib.colors as mcol

unit = u.ct

fig, ax = plt.subplots()
norm = mcol.LogNorm()
cmap = plt.get_cmap('plasma').copy()
ax.pcolormesh(
    time_bins.to_value(u.s),
    energy_bins,
    data.T.value,
    norm=norm,
    cmap=cmap
)
ax.set(yscale='log', xlabel='time (s)', ylabel='energy (keV)', title='noisy spectrogram')
plt.show()

In [ ]:
from yaff import plotting
fig, ax = plt.subplots()
def pois(a):
    a = a.to_value(u.ct)
    return np.sqrt(a) << u.ct

with viz.quantity_support():
    plotting.stairs_with_error(energy_bins, th := thermal_truth.sum(axis=0), pois(th), ax=ax, label='data')
    plotting.stairs_with_error(energy_bins, nth := nonthermal_truth.sum(axis=0), pois(nth), ax=ax, label='data')
    plotting.stairs_with_error(energy_bins, bkg := noise_spectrogram.sum(axis=0), pois(bkg), ax=ax, label='data')
    ax.set(xscale='log', yscale='log')
    ax.legend()
plt.show()

In [ ]:
nearest = lambda a, v: np.argmin(np.abs(a - v))

fig, ax = plt.subplots()
energy_bounds = ((4, 10), (10, 20), (20, 40), (35, 47), (40, 80), (80, 300)) << u.keV
for (ea, eb) in energy_bounds:
    a, b = nearest(energy_mids, ea), nearest(energy_mids, eb)
    s = data[:, a:b].sum(axis=1)
    plotting.stairs_with_error(time_bins, s, pois(s), label=f"{ea:.0f} $\\rightarrow$ {eb:.0f}")
plotting.stairs_with_error(time_bins, data[:, -1], label="bkg")
ax.set(yscale='log', ylim=(1e-9, 1e7))
ax.legend()
plt.show()

In [ ]:
from tedec import decomp

In [ ]:
dimensionless_data = data.T.astype(float)
dimensionless_errors = data_err.T.astype(float)

thermal_idx = nearest(energy_mids, 5 << u.keV)
nonthermal_idx = nearest(energy_mids, 22 << u.keV)

pack = decomp.DataPacket(
    data=dimensionless_data,
    basis_timeseries=[
        dimensionless_data[thermal_idx - 1 : thermal_idx + 2].sum(axis=0),
        dimensionless_data[nonthermal_idx - 1 : nonthermal_idx + 2].sum(axis=0),
        dimensionless_data[-1]
    ],
    constant_offset=False
)

# systematic = 0.1
ret = decomp.bootstrap(
    pack,
    errors=dimensionless_errors,
    num_iter=1000
)

In [ ]:
from yaff import plotting
from astropy import visualization as viz

sys = 0.05
err = lambda a: np.sqrt(a + (a * sys)**2)

th_mean = ret[:, 0, :].mean(axis=0) << u.ct
th_std = ret[:, 0, :].std(axis=0) << u.ct
nth_mean = ret[:, 1, :].mean(axis=0) << u.ct
nth_std = ret[:, 1, :].std(axis=0) << u.ct

# scale by # time bins (need to update)
bkg_part = ret[:, 2, :]# * (num_time_bins - 1)
# bkg_part = ret[:, 1, :] * 0
bkg_mean = bkg_part.mean(axis=0) * u.ct
bkg_std = bkg_part.std(axis=0) * u.ct

fig, ax = plt.subplots()

with viz.quantity_support():
    # plotting.stairs_with_error(energy_bins, th := thermal_truth.sum(axis=0), err(th.value) << u.ct, label='true thermal spectrum')
    plotting.stairs_with_error(energy_bins, nth := nonthermal_truth.sum(axis=0), err(nth.value) << u.ct, label='true nonthermal spectrum')
    # plotting.stairs_with_error(energy_bins, dm := dimensionless_data.sum(axis=1) << u.ct, err(dm.value) << u.ct, label='full spectrum')
    ax.stairs(noise_spectrogram.sum(axis=0), energy_bins, label='true background')
    
    num_sigma = 2
    # plotting.stairs_with_error(energy_bins << u.keV, th_mean << u.ct, num_sigma*th_std, ax=ax, label='decomposed thermal')
    plotting.stairs_with_error(energy_bins << u.keV, nth_mean << u.ct, num_sigma*nth_std, ax=ax, label='decomposed nonthermal')
    plotting.stairs_with_error(energy_bins << u.keV, bkg_mean << u.ct, num_sigma*bkg_std, ax=ax, label='decomposed background')
    plotting.stairs_with_error(
        energy_bins << u.keV,
        (nth_mean + th_mean + bkg_mean) << u.ct,
        num_sigma*np.sqrt(nth_std**2 + th_std**2 + bkg_std**2),
        ax=ax,
        label='decomposed full'
    )

ax.legend()
ax.set(xscale='log', yscale='log', ylim=(None, None))
plt.show()

In [ ]:
import fathon
from fathon import fathonUtils as fu

hursts = list()
for curve in data.T:
    curve = curve.copy()
    # a = fu.toAggregated(curve)
    a = fu.subtractMean(curve)
    dfa = fathon.DFA(a)
    min_window = 4
    windows = np.arange(min_window, a.size)
    n, f = dfa.computeFlucVec(windows, revSeg=True, polOrd=2)
    h, h_intercept = dfa.fitFlucVec()
    hursts.append(h)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(1 + np.arange(len(hursts)), hursts)


ax.axhspan(1, 10, color='magenta', alpha=0.1, zorder=-1, label='nonstationary')
ax.axhspan(0.5, 1, color='green', alpha=0.1, zorder=-1, label='persistent')
ax.axhspan(0., 0.5, color='blue', alpha=0.1, zorder=-1, label='antipersistent')
ax.set(xlabel='Energy', ylabel='Hurst exponent of light curve', title='Fractal analysis of simulated series', ylim=(0, 2.1), xscale='log')

In [ ]:
from yaff import fitting
from yaff import common_likelihoods

systematic = lambda s, c, a: np.sqrt(s**2 + ((c * a).value << s.unit)**2)

sys = 0.05
nth_data = fitting.DataPacket(
    counts=(nth_as_cts := nth_mean.to_value(u.ct) << u.ct),
    counts_error=systematic(nth_std, nth_as_cts, sys).to_value(u.ct) << u.ct,
    background_counts=(0 * nth_as_cts),
    background_counts_error=(0 * nth_as_cts),
    effective_exposure=total_time,
    count_energy_edges=energy_bins,
    photon_energy_edges=energy_bins,
    response_matrix=np.diag(response_vector),#area * (np.eye(nth_as_cts.size) << (u.ct / u.ph))
)

nonthermal_priors = {
    'electron_flux': fitting.simple_bounds(0, 20),
    'spectral_index': fitting.simple_bounds(2.1, 10),
    'cutoff_energy': fitting.simple_bounds(5, 80),
}

nonthermal_params = {
    'electron_flux': fitting.Parameter(2 << (1e35 * u.electron / u.s), False),
    'spectral_index': fitting.Parameter(5 << u.one, False),
    'cutoff_energy': fitting.Parameter(20 << u.keV, True),
}

rng = np.random.default_rng()
params = {
    k: fitting.Parameter(v.as_quantity() * rng.uniform(0.9, 1.1), frozen=False)
    for (k, v) in nonthermal_params.items()
}
params['cutoff_energy'].value = 10
params['cutoff_energy'].frozen = True

fit_range = (energy_mids.value > 12)
likelihood = common_likelihoods.chi_squared_factory(fit_range)

fr = fitting.BayesFitter(
    data=nth_data,
    model_function=cm.thick_target,
    parameters=params,
    log_priors=nonthermal_priors,
    log_likelihood=likelihood
)

In [ ]:
fr = fitting.levenberg_minimize(fr)
fr.parameters

In [ ]:
fr.run_emcee(
    emcee_constructor_kw={'nwalkers': 20},
    emcee_run_kw={'nsteps': 1000}
)

In [ ]:
from yaff import plotting as yap
yap.plot_parameter_chains(
    fr,
    names=fr.free_param_names,
    params=list(fr.free_parameters)
)
plt.show()

In [ ]:
samples = fr.generate_model_samples(100)
fig = plt.figure()
yap.plot_data_model(fr, model_samples=samples, fig=fig)
plt.show()

In [ ]:
import corner

burnin = (50 * fr.emcee_sampler.nwalkers)
corner_chain = fr.emcee_sampler.flatchain[burnin:]
param_names = fr.free_param_names

fig = plt.figure(figsize=(10, 8), layout="tight")
corner.corner(
    corner_chain,
    fig=fig,
    bins=20,
    labels=param_names,
    quantiles=(0.05, 0.5, 0.95),
    show_titles=True,
    truths=(
        low_eflux.mean(),
        low_idx.mean(), 
#         20
    ),
    # plot_contours=False,
    # range=(
    #     (14, 20),
    #     (3.9, 4.4),
    #     (24, 30)
    # ),
    truth_color='red'
)
fig.savefig('decomp nonthermal.png', dpi=300)
plt.show()

In [ ]:
low_eflux.mean(), low_idx.mean(), low_eflux.min(), low_eflux.max()
# low_idx.min(), low_idx.max()

### Fit thermal decomposed data

In [ ]:
from yaff import fitting
from yaff import common_likelihoods

th_data = fitting.DataPacket(
    counts=(th_as_cts := th_mean),
    counts_error=systematic(th_std, th_as_cts, sys) << u.ct,
    background_counts=(0 * th_as_cts),
    background_counts_error=(0 * th_as_cts),
    effective_exposure=total_time,
    count_energy_edges=energy_bins,
    photon_energy_edges=energy_bins,
    response_matrix=np.diag(response_vector)
)

thermal_priors = {
    'temperature': fitting.simple_bounds(10, 40),
    'emission_measure': fitting.simple_bounds(1e-4, 1e4),
}

thermal_params = {
    'temperature': fitting.Parameter(20 << u.MK, False),
    'emission_measure': fitting.Parameter(1 << (1e47 * u.cm**-3), False),
}

rng = np.random.default_rng()
params = {
    k: fitting.Parameter(v.as_quantity() * rng.uniform(0.9, 1.1), frozen=False)
    for (k, v) in thermal_params.items()
}

likelihood = common_likelihoods.chi_squared_factory(
    restriction=(restriction := (energy_mids.value < 20))
)

fr = fitting.BayesFitter(
    data=th_data,
    model_function=cm.thermal,
    parameters=params,
    log_priors=thermal_priors,
    log_likelihood=likelihood
)

In [ ]:
fitting.levenberg_minimize(fr, restriction=restriction)

In [ ]:
fr.parameters

In [ ]:
fr.run_emcee(
    emcee_constructor_kw={'nwalkers': 20},
    emcee_run_kw={'nsteps': 1000}
)


In [ ]:
from yaff import plotting as yap
yap.plot_parameter_chains(
    fr,
    names=fr.free_param_names,
    params=list(fr.free_parameters)
)
plt.show()

In [ ]:
import corner

burnin = (100 * fr.emcee_sampler.nwalkers)
corner_chain = fr.emcee_sampler.flatchain[burnin:]
param_names = fr.free_param_names

fig = plt.figure(figsize=(10, 8), layout="tight")
corner.corner(
    corner_chain,
    fig=fig,
    bins=20,
    labels=param_names,
    quantiles=(0.05, 0.5, 0.95),
    show_titles=True,
    truths=(
        temps.mean().to_value(u.MK),
        em_start.value,
    ),
    truth_color='red'
)
plt.savefig('decomp thermal.png', dpi=300)
plt.show()

In [ ]:
temps.min(), temps.max()

In [ ]:
samples = fr.generate_model_samples(num=100)
fig = plt.figure()
yap.plot_data_model(fr, model_samples=samples, fig=fig)
plt.show()

## Do a traditional two-model fit

In [ ]:
dp = fitting.DataPacket(
    counts=(cts := data.sum(axis=0).value) << u.ct,
    counts_error=np.sqrt(cts + (sys * cts)**2) << u.ct,
    background_counts=(bg := noise_spectrogram.sum(axis=0).value) << u.ct,
    background_counts_error=np.sqrt(bg + (sys * bg)**2) << u.ct,
    effective_exposure=total_time,
    count_energy_edges=energy_bins,
    photon_energy_edges=energy_bins,
    response_matrix=np.diag(response_vector)
)

In [ ]:
priors = thermal_priors | nonthermal_priors
params = {
    k: fitting.Parameter(v.as_quantity() * rng.uniform(0.9, 1.1), frozen=False)
    for (k, v) in (thermal_params | nonthermal_params).items()
}

def model(args):
    return cm.thermal(args) + cm.thick_target(args)

likelihood = common_likelihoods.chi_squared_factory(restriction := energy_mids.value < 70)

fr = fitting.BayesFitter(
    data=dp,
    model_function=model,
    parameters=params,
    log_priors=priors,
    log_likelihood=likelihood
)

In [ ]:
fr.parameters

In [ ]:
fitting.levenberg_minimize(fr, restriction)

In [ ]:
fr.parameters

In [ ]:
fr.run_emcee(
    emcee_constructor_kw={'nwalkers': 20},
    emcee_run_kw={'nsteps': 1000}
)


In [ ]:
from yaff import plotting as yap
yap.plot_parameter_chains(
    fr,
    names=fr.free_param_names,
    params=list(fr.free_parameters.values())
)
plt.show()

In [ ]:
fr.parameters

In [ ]:
import corner

burnin = (50 * fr.emcee_sampler.nwalkers)
corner_chain = fr.emcee_sampler.flatchain[burnin:]
param_names = fr.free_param_names

fig = plt.figure(figsize=(20, 20), layout="tight")
corner.corner(
    corner_chain,
    fig=fig,
    bins=20,
    labels=param_names,
    quantiles=(0.05, 0.5, 0.95),
    show_titles=True,
    truths=(
        temps.mean().to_value(u.MK),
        em_start.value,
        low_eflux.mean(),
        low_idx.mean(), 
        20,
    ),
    truth_color='red'
)

plt.savefig('traditional.png', dpi=300)
plt.show()

In [ ]:
samples = fr.generate_model_samples(num=100)
fig = plt.figure()
yap.plot_data_model(fr, model_samples=samples, fig=fig)
plt.show()

In [ ]:
low_idx.mean(), low_idx.std()